# Select GPU

Runtime -> Change Runtime type -> T4 GPU

# Check nvidia

## install if needed

In [ ]:
!pip install --upgrade nvidia-pyindex

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8419 sha256=9c77ce6b59f64da924cb407482ae7f84ba351c73c8f2edc6c8ad4efc81f532f6
  Stored in directory: /root/.cache/pip/wheels/2c/af/d0/7a12f82cab69f65d51107f48bcd6179e29b9a69a90546332b3
Successfully built nvidia-pyindex


## check

In [ ]:
!nvidia-smi

Mon Jun 17 05:17:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Packages

## Installation

### For numpy version issue, no need if this issue is solved

In [ ]:
!pip install numpy==1.23.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 27.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.1 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.1 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.23.1 which is incompatible.


In [ ]:
import numpy as np
np.version.version

'1.23.1'

### For notebook import

In [ ]:
!pip install import_ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.3 MB/s eta 0:00:00


#Mount Drive

In [ ]:
# import packages
## for mount drive purpose
import os
from google.colab import drive
# mount drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/Colab_Notebooks/TSPE/')

Mounted at /content/drive/


# packages

In [ ]:
from utils import get_gene_idx_dict_from_file, file_to_matrix
import pickle
import numpy as np

import torch
import torch.utils.data as data

import import_ipynb
from training_helper import transformerGO_collate_fn

from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
import torch.nn as nn

from sklearn.metrics import roc_auc_score
from sklearn import metrics

import numpy as np
import time

from tqdm import tqdm as prog_bar #The progress bar

import import_ipynb
from transformerGO import (TransformerGO_Scratch)
from training_helper import (print_status, write_scalars_tensorboard,
                             binary_accuracy, epoch_time)

# Set info


In [ ]:
input_folder = 'input'
dataset = 'RR0'   # 'RR1'
emb_folder = "embedding"

# PSE = 'NoPE' # opt0
# PSE = 'LPE' # opt1
# PSE = 'SPE' # opt2

node_file_path = f'{input_folder}/interactom_nodes.txt'   # stores the nodes for the largest connected component in human Interactome
train_file_path = f'{input_folder}/{dataset}/train_set.tsv'
test_file_path = f'{input_folder}/{dataset}/test_set.tsv'

# Prep Data

In [ ]:
def get_disease_sets(file_path):
    dis_pairs = []   #[(disA, disB), ...]
    labels = []      # [label, ...]
    disease_genes_dict = {}     #{disease: [gene_1, gene_2, ...]}

    f = open(file_path, "r")
    head = True
    for line in f:
        if head:
            head = False
            continue

        row = line.strip().split("\t")
        dis_pair, disease_a_genes, disease_b_genes, all_genes, rr = row

        disease_a, disease_b = dis_pair.split("&")

        dis_pairs.append((disease_a, disease_b))
        labels.append(int(rr))

        disease_genes_dict[disease_a] = disease_a_genes.split(",")
        disease_genes_dict[disease_b] = disease_b_genes.split(",")


    f.close()

    return dis_pairs, labels, disease_genes_dict

# TransformerGO: (np.array(emb_protA), np.array(emb_protB)), label, ((protA, protein_go_anno[protA]), (protB, protein_go_anno[protB]))
# Mine: (np.array(emb_disA), np.array(emb_disB)), label, ((disA, disA_gene_list), (disB, disB_gene_list))

class Dataset_torch(torch.utils.data.Dataset):
    #Characterizes a dataset for PyTorch
    def __init__(self, dis_pairs, labels, dis_genes_dict, node_idx_dict, emb, lpe, combine_opt):
        self.dis_pairs = dis_pairs
        self.labels = labels
        self.dis_genes_dict = dis_genes_dict
        self.node_idx_dict = node_idx_dict
        self.emb = emb
        self.lpe = lpe
        self.opt = combine_opt

    def __len__(self):
        return len(self.dis_pairs)

    def __getitem__(self, index):

        label = self.labels[index]
        disA,disB = self.dis_pairs[index]
        gene_lists = [self.dis_genes_dict[disA], self.dis_genes_dict[disB]]
        dis_pair_genes = [(disA, gene_lists[0]), (disB, gene_lists[1])] # [(disA, disA_gene_list), (disB, disB_gene_list)]
        features = [get_features(gene_list, node_idx_dict, self.emb, self.lpe, self.opt) for gene_list in gene_lists]

        return np.array((features, label, dis_pair_genes), dtype=object)

#------------------------------------------------------------------------------#
def get_features(gene_list, node_idx_dict, emb, lpe, combine_opt):
    node_idices = [node_idx_dict[gene] for gene in gene_list if gene in node_idx_dict]
    feature_vecs = emb[node_idices, :]
    # print(type(feature_vecs))
    # print(feature_vecs.shape)
    # print(type(lpe[node_idices, :]))
    # print(lpe[node_idices, :].shape)
    # print(type(np.concatenate(emb[node_idices, :], lpe[node_idices, :], axis=1)))
    # print(np.concatenate(emb[node_idices, :], lpe[node_idices, :], axis=1).shape)
    if lpe is not None:
      if combine_opt == "add":
        feature_vecs = np.add(emb[node_idices, :], lpe[node_idices, :])
      else:
        feature_vecs = np.concatenate((emb[node_idices, :], lpe[node_idices, :]), axis=1)

    return feature_vecs

#------------------------------------------------------------------------------#
def split_train_valid(ori_dataset, ratio):
  sz = len(ori_dataset)
  train_set, valid_set = data.random_split(ori_dataset, [int(ratio[0]*sz), sz - (int(ratio[0]*sz)) ] )

  return train_set, valid_set

########################TransformerGO###########################################
def get_max_len_seq(dataset):
    """Finds the dis with the most genes and returns the size"""
    batch_features, batch_labels, batch_ids  = zip(*dataset)
    batch_features = np.array(batch_features)


    max_len = 0
    for i in range(0, batch_features.shape[0]):
        max_len = max(max_len, len(batch_features[i][0]), len(batch_features[i][1]))
    return max_len

def helper_collate(batch):
    MAX_LEN_SEQ = get_max_len_seq(batch)
    return transformerGO_collate_fn(batch, MAX_LEN_SEQ, EMB_DIM, pytorch_pad = False)

def train(model, iterator, optimizer, criterion,  torch_vers = False):

    epoch_loss = 0
    epoch_acc = 0
    model.train()

    pred = []
    lab = []
    for batch in prog_bar(iterator):
        optimizer.zero_grad()

        #padded pairs: tensor of shape N * 2(protein pair) * L(longest seq) * Emb dim
        padded_pairs = batch[0].to(device)
        labels = batch[1].to(device)
        mask = batch[2].to(device)

        #split data into protA and protB
        gosetA_batch = padded_pairs[:,0]
        gosetB_batch = padded_pairs[:,1]

        #permute the data to fit the pytorch transformer
        if torch_vers:
            gosetA_batch = gosetA_batch.permute(1,0,2)
            gosetB_batch = gosetB_batch.permute(1,0,2)

        predictions = model(gosetA_batch, gosetB_batch, mask[:,0], mask[:,1]).squeeze(1)
        loss = criterion(predictions, labels)
        acc = binary_accuracy(predictions, labels)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

        pred = pred + list(predictions.cpu().data.numpy())
        lab = lab + list(labels.cpu().data.numpy())

    return epoch_loss / len(iterator), epoch_acc / len(iterator), roc_auc_score(lab,pred)

def evaluate(model, iterator, criterion, torch_vers = False):

    epoch_loss = 0
    epoch_acc = 0
    pred = []
    lab = []

    model.eval()
    with torch.no_grad():
        for batch in iterator:

            #padded pairs: tensor of shape N * 2(protein pair) * L(longest seq) * Emb dim
            padded_pairs = batch[0].to(device)
            labels = batch[1].to(device)
            mask = batch[2].to(device)

            #split data into protA and protB
            gosetA_batch = padded_pairs[:,0]
            gosetB_batch = padded_pairs[:,1]

            #permute the data to fit the pytorch transformer
            if torch_vers:
                gosetA_batch = gosetA_batch.permute(1,0,2)
                gosetB_batch = gosetB_batch.permute(1,0,2)

            predictions = model(gosetA_batch, gosetB_batch, mask[:,0], mask[:,1]).squeeze(1)
            loss = criterion(predictions, labels)
            acc = binary_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

            pred = pred + list(predictions.cpu().data.numpy())
            lab = lab + list(labels.cpu().data.numpy())

    return epoch_loss / len(iterator), epoch_acc / len(iterator), roc_auc_score(lab,pred), lab, pred

def evaluate_cosine(iterator, criterion, torch_vers = False):

    epoch_loss = 0
    epoch_acc = 0
    pred = []
    lab = []

    for batch in prog_bar(iterator):

        #padded pairs: tensor of shape N * 2(protein pair) * L(longest seq) * Emb dim
        padded_pairs = batch[0].to(device)
        labels = batch[1].to(device)

        #split data into protA and protB
        gosetA_batch = padded_pairs[:,0]
        gosetB_batch = padded_pairs[:,1]

        vector_protA = torch.sum(gosetA_batch, dim = 1)
        vector_protB = torch.sum(gosetB_batch, dim = 1)

        cosine = nn.CosineSimilarity(dim=1, eps=1e-6)
        predictions = cosine(vector_protA, vector_protB)
        loss = criterion(predictions, labels)
        acc = binary_accuracy(predictions, labels)

        epoch_loss += loss.item()
        epoch_acc += acc.item()

        pred = pred + list(predictions.cpu().data.numpy())
        lab = lab + list(labels.cpu().data.numpy())

    return epoch_loss / len(iterator), epoch_acc / len(iterator), lab, pred

In [ ]:
# 1. get graph original nodes
node_idx_dict = get_gene_idx_dict_from_file(node_file_path)
node_gene_dict = {v:k for k,v in node_idx_dict.items()}

# 2. get selected disease pairs
train_dis_pairs, train_labels, train_disease_genes_dict = get_disease_sets(train_file_path)
test_dis_pairs, test_labels, test_disease_genes_dict = get_disease_sets(test_file_path)

# 3. get embedding file
#### opt0: n2v from graph with id mapping, 64 d
if PSE == 'NoPE':
  emb_dim = 64
  lpe_dim = 0
  emb_file = f'{emb_folder}/node2nev_emb_64' # f'{emb_folder}/node2nev_emb_64'
  with open(emb_file, 'rb') as f:
      emb = pickle.load(f)
  lpe = None
  combine_opt = None

#### opt1: n2v from graph with id mapping + LPE Add
if PSE == 'LPE':
  emb_dim = 64
  lpe_dim = 64
  emb_file = f'{emb_folder}/node2nev_emb_64'
  with open(emb_file, 'rb') as f:
      emb = pickle.load(f)

  lpe_file = f'{emb_folder}/LPE.tsv'
  lpe = file_to_matrix(lpe_file)
  combine_opt = "add"

#### opt3: n2v from graph with id mapping + LPE add + GPE concat
if PSE == 'SPE':
  emb_dim = 64
  lpe_dim = 64

  emb_file = f'{emb_folder}/node2nev_emb_64'
  with open(emb_file, 'rb') as f:
      emb = pickle.load(f)

  # --add lpe
  lpe_file = f'{emb_folder}/LPE.tsv'
  lpe = file_to_matrix(lpe_file)
  emb = np.add(emb, lpe)

  # --concat Gpe later
  lpe_dim = 8

  lpe_file = f'{emb_folder}/GPE.tsv'
  lpe = file_to_matrix(lpe_file)
  lpe = lpe[:,1:lpe_dim+1]
  combine_opt = "concat"


train_origin_set = Dataset_torch(train_dis_pairs, train_labels, train_disease_genes_dict, node_idx_dict, emb, lpe, combine_opt)
ratio = [0.9, 0.1] # [0.8, 0.2]  # [0.9, 0.1]
train_set, valid_set = split_train_valid(train_origin_set, ratio)
test_set = Dataset_torch(test_dis_pairs, test_labels, test_disease_genes_dict, node_idx_dict, emb,  lpe, combine_opt)

params = {'batch_size': 20,'collate_fn': helper_collate}

print("Train set: ", len(train_set), '\n', "Valid set: ", len(valid_set), '\n', "Test set: ", len(test_set), '\n')
train_grt = data.DataLoader(train_set, **params, shuffle = True)
val_grt = data.DataLoader(valid_set, **params, shuffle = True)
test_grt = data.DataLoader(test_set, **params, shuffle = False)


if lpe_dim == 64:
  EMB_DIM = emb_dim
else: EMB_DIM = emb_dim + lpe_dim  # N2V concate LPE, 64 *2 dimensions too much, cannot run

print(f"emb dim: {EMB_DIM}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device available: ", device, " ", torch.cuda.get_device_name(0))

#TRANSFORMERGO MODEL #
MODEL_SIZE = EMB_DIM
NR_HEADS = 8
NR_LAYERS = 3
DROPOUT = 0.2
SIZE_FF = 4 * MODEL_SIZE
LR = 0.0001

# if see AssertionError, this is because the dimension (EMB_DIM) cannot divide the headnumber (NR_HEADS)
# for the current one, add 8 dim for gpe and keep original 64 dims
model = TransformerGO_Scratch(MODEL_SIZE, NR_HEADS, NR_LAYERS, SIZE_FF, DROPOUT)

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.BCEWithLogitsLoss().to(device) # this is the activation function used

pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)


In [ ]:
model_name = "model.pt"

writer = SummaryWriter(flush_secs=14)
N_EPOCHS = 30
best_roc_val = float('-inf')

for epoch in range(N_EPOCHS):


    start_time = time.time()
    train_loss, train_acc, roc_train = train(model, train_grt, optimizer, criterion, torch_vers = False)
    valid_loss, valid_acc, roc_val, _, _ = evaluate(model, val_grt, criterion, torch_vers = False)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if best_roc_val < roc_val:
        best_roc_val = roc_val
        torch.save(model.state_dict(),  model_name)

    print_status(epoch, epoch_mins, epoch_secs, train_loss,\
                 train_acc, valid_loss, valid_acc, roc_train, roc_val, optimizer)
    write_scalars_tensorboard(writer, train_loss, valid_loss, train_acc, valid_acc, epoch)

In [ ]:
#WRiTING THE PERFORMANCE ON THE TEST SET #
model = TransformerGO_Scratch(MODEL_SIZE, NR_HEADS, NR_LAYERS, SIZE_FF, DROPOUT)
model.load_state_dict(torch.load(model_name))
model = model.to(device)

EMB_method = 'N2V'

with open(f"{dataset}_{fold}_{EMB_method}_{PSE}_training-results.txt", "a") as myfile:
    myfile.write(f"\n ### {model_name} ### \n")
    myfile.write(f"\n ### EMB: {EMB_method}; DIM: {EMB_DIM} ### \n")
    myfile.write(f"Train set: {len(train_set)}, Valid set: {len(valid_set)}, Test set: {len(test_set)} \n")

    valid_loss, valid_acc, roc_val, lab, pred = evaluate(model, test_grt, criterion, torch_vers = False)
    myfile.write(f" \n valid_loss: {valid_loss}, valid_acc: {valid_acc}, roc_val: {roc_val} \n")

# Disconnect when done

In [ ]:
!kill $(ps aux | awk '{print $2}')